# Converting Take All Documents into JSON

## Documentation

This Jupyter Notebook takes in translations of the Take One brochure and outputs it as a JSON file for the MyBus tool.

The data was originally in a Word document.  In transferring it to a Word document, line breaks and spaces were cleaned up in the content.  Different languages use spaces differently.

The output file is used on the "All Changes" page of the MyBus tool to display the Take One brochure as an HTML page instead of only as a PDF file.  It contains all the details for all line changes aggregated into a single view.

### Notes

#### Not All Lines

Not all lines are listed in the Take One brochure, only those with major changes.  Some lines not listed in the brochure will still have updated schedules due to minor changes.  For the All Changes page to also act as a central source for updated schedule PDFs, this data needed to be updated.

#### Line Numbers

Lines with sister routes are listed in the brochure as a combined line.  For example - the 16/17.  To match entries with their corresponding schedule PDFs, an additional field for the line number was added.


## Setup 
### 1.1 Import modules

In [67]:
import pandas as pd 
import numpy as np
from docx.api import Document
# import re
# import json

# templates = [["header",1,"Metro is making more service changes.","Metro está haciendo más cambios en sus servicios.","Metro正在進行更多服務調整。","Metro hiện đang thực hiện nhiều thay đổi về dịch vụ.","메트로 서비스가 더욱 새롭게단장하고 있습니다.","メトロのサービスが変更されます。","Metro-ն կրկին փոփոխություններ է իրականացնում ծառայությունների մեջ:","Metro вносит дополнительные изменения в схемы движения."]]
# templates = ["header",1],["summary",1],["details",1],["end",1]
# final_template = pd.DataFrame(templates,columns=["section","order","en","es","zh-TW","vi","ko","ja","hy","ru"])


### 1.2 Read .docx and set final output

In [68]:
document = Document('../data/input/202109shakeup.docx')
table = document.tables[0]

headers = ["section","order","line","altline","en","es","zh-TW","vi","ko","ja","hy","ru","new-schedule","current-schedule"]

def reset_final_df():
    return pd.DataFrame(columns=headers)

final_df = pd.DataFrame(columns=headers)

### 1.3 Set dataframe to docx table and pre-process data

In [69]:
document = Document('../data/input/202109shakeup.docx')
table = document.tables[0]
data = [[cell.text.replace("\n"," ").replace('"','').replace('" ','').lstrip() for cell in row.cells] for row in table.rows]

df = pd.DataFrame(data)
new_header = df.iloc[0]
df = df[1:] 
df.columns = new_header
# print(df.columns)
df = df.rename(columns={'English':'en','Spanish':'es','Chinese (Traditional)':'zh-TW','Korean':'ko','Vietnamese':'vi','Japanese':'ja','Russian':'ru','Armenian':'hy'})
# df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)

df = df.replace(' +',r' ',regex=True)
df = df.replace('"',r'',regex=True)
# df.to_json('test.json')
# df.to_csv('test.csv')
df.head()

final_df = pd.DataFrame(columns=["section","order","line","altline","en","es","zh-TW","vi","ko","ja","hy","ru","new-schedule","current-schedule"])

## Populating the data

### 2.1 Adding the `Summary` sections

In [70]:
header1 = df.loc[(df['en'].str.contains('\u2013') == False) & (df['en'].str.contains('Metro is making service'))]
header1 = header1.assign(section='header')
header1 = header1.assign(order='1')

header2 = df.loc[(df['en'].str.contains('\u2013') == False) & (df['en'].str.contains('New schedules start'))]
header2 = header2.assign(section='header')
header2 = header2.assign(order='2')

if not final_df.empty:
    final_df = reset_final_df()

final_df = final_df.append(header1)
final_df = final_df.append(header2)

final_df

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,new-schedule,current-schedule
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN,NaN


### 2.1.1 Populating the `Summary` sections

In [71]:
# th = df[df['en'].str.contains('Starting on'):df['en'].str.contains('We’re ')]
th = df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 30) & (df['en'].str.contains('We’re modify') == False)]

th = th.assign(section='summary')

th['order'] = ''

th_count = th.shape[0]
for i in range(0,th_count):
    th['order'].values[i] = i

th

final_df = final_df.append(th)
final_df

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,new-schedule,current-schedule
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN,NaN
1,summary,0,NaN,NaN,"Starting on Sunday, September 12, 2021, Metro ...","A partir del domingo, 12 de septiembre de 2021...",自2021年9月12日週日開始， Metro將對服務進行變更， 以便為您帶來更好的巴士 體驗。,"Bắt đầu từ Chủ Nhật, ngày 12 tháng 9 năm 2021,...","2021년 9월 12일, 일요일부터 개시, Metro가 향상된 버스 경험을 제공해 ...",2021年9月12日 日曜日より、 Metroは お客様により良いバスの旅をご体験いただける...,"2021 թվականի կիրակի, սեպտեմբերի 12-ից սկսած, M...","Начиная с субботы, 12 сентября 2021 года, Metr...",NaN,NaN
2,summary,1,NaN,NaN,We’re realigning routes to better match travel...,Estamos reorganizando las rutas para que se aj...,我們將重新調整行駛路線， 以讓其與乘車模式更加匹配， 並且會增加出行路線，以便實施 Next...,Chúng tôi sẽ điều chỉnh lại các tuyến lộ trình...,더 원할한 통행 패턴 반영을 위한 노선 재편과 운행 증편을 통해 NextGen 버스...,もっと旅行パターンに合わせたルートの再編成を行い、 次世代バスプラン (NextGen Bu...,Մենք փոփոխություններ ենք կատարում երթուղիներու...,"Мы изменяем схемы движения, чтобы привести их ...",NaN,NaN
3,summary,2,NaN,NaN,Some bus stops have been removed to improve tr...,Se eliminaron algunas paradas de autobús para ...,為縮短乘車時間已停用部分巴士站。,Một số trạm dừng xe buýt đã bị xóa bỏ để cải t...,운행 시간 개선을 위해 일부 버스 정류소를 폐쇄하였습니다.,いくつかのバス停は移動時間を改善するために撤去されました。,Որոշ ավտոբուսային կանգառներ հանվել են` ճանապար...,"Некоторые автобусные остановки отменены, чтобы...",NaN,NaN
4,summary,3,NaN,NaN,The following lines will have additional trips...,Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日、 週六及週日提供額外的出行服務： 2, 4, 16, 33, 45, 5...",Các tuyến sau đây sẽ có thêm chuyến vào các ng...,"다음 노선에는 주중, 토요일과 일요일에 운행을 증편할 예정입니다: 2, 4, 16,...","次の路線は、 平日、土曜日、日曜日の本数が追加されます: 2, 4, 16, 33, 45,...",Հետևյալ գծերը կունենան լրացուցիչ ուղևորություն...,По следующим маршрутам вводятся дополнительные...,NaN,NaN
5,summary,4,NaN,NaN,"WEEKDAYS, the following lines will have additi...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日提供 額外的出行服務： 10, 14, 28, 40, 51, 53, 7...","VÀO CÁC NGÀY TRONG TUẦN, những tuyến sau sẽ đư...","주중에 다음 노선은 운행을 증편할 예정입니다: 10, 14, 28, 40, 51, ...","平日は、次の路線の本数が 追加されます: 10, 14, 28, 40, 51, 53, 7...","ԱՇԽԱՏԱՆՔԱՅԻՆ ՕՐԵՐԻՆ, հետևյալ գծերը կունենան լր...",По БУДНЯМ по следующим маршрутам будут осущест...,NaN,NaN
6,summary,5,NaN,NaN,"SATURDAYS and SUNDAYS, the following lines hav...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在週六及週日提供 額外的出行服務： 154, 155, 210","VÀO THỨ BẢY và CHỦ NHẬT, những tuyến sau sẽ đư...","토요일과 일요일에 다음 노선은 운행을 증편할 예정입니다: 154, 155, 210","土曜日、日曜日は、 次の路線の本数が追加されます: 154, 155, 210","ՇԱԲԱԹ և ԿԻՐԱԿԻ օրերին, հետևյալ գծերն ունեն լրա...",По СУББОТАМ и ВОСКРЕСЕНЬЯМ вводятся дополнител...,NaN,NaN


### 2.1.2 Adding Metro Rail Lines in the summary section

In [72]:
### filter out the rail lines
### note: right now this is hard coded... need a list of rail lines..
rail_df = df.loc[(df['en'].str.contains('\u2013')) & (df['en'].str.contains('B Line, D Line') == True)]

### add this to the end of all the lines
end_lines = len(th) +1

### set the properties
rail_df = rail_df.assign(section='summary')
rail_df = rail_df.assign(order=end_lines)

### add to the final data frame
final_df = final_df.append(rail_df)
final_df

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,new-schedule,current-schedule
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN,NaN
1,summary,0,NaN,NaN,"Starting on Sunday, September 12, 2021, Metro ...","A partir del domingo, 12 de septiembre de 2021...",自2021年9月12日週日開始， Metro將對服務進行變更， 以便為您帶來更好的巴士 體驗。,"Bắt đầu từ Chủ Nhật, ngày 12 tháng 9 năm 2021,...","2021년 9월 12일, 일요일부터 개시, Metro가 향상된 버스 경험을 제공해 ...",2021年9月12日 日曜日より、 Metroは お客様により良いバスの旅をご体験いただける...,"2021 թվականի կիրակի, սեպտեմբերի 12-ից սկսած, M...","Начиная с субботы, 12 сентября 2021 года, Metr...",NaN,NaN
2,summary,1,NaN,NaN,We’re realigning routes to better match travel...,Estamos reorganizando las rutas para que se aj...,我們將重新調整行駛路線， 以讓其與乘車模式更加匹配， 並且會增加出行路線，以便實施 Next...,Chúng tôi sẽ điều chỉnh lại các tuyến lộ trình...,더 원할한 통행 패턴 반영을 위한 노선 재편과 운행 증편을 통해 NextGen 버스...,もっと旅行パターンに合わせたルートの再編成を行い、 次世代バスプラン (NextGen Bu...,Մենք փոփոխություններ ենք կատարում երթուղիներու...,"Мы изменяем схемы движения, чтобы привести их ...",NaN,NaN
3,summary,2,NaN,NaN,Some bus stops have been removed to improve tr...,Se eliminaron algunas paradas de autobús para ...,為縮短乘車時間已停用部分巴士站。,Một số trạm dừng xe buýt đã bị xóa bỏ để cải t...,운행 시간 개선을 위해 일부 버스 정류소를 폐쇄하였습니다.,いくつかのバス停は移動時間を改善するために撤去されました。,Որոշ ավտոբուսային կանգառներ հանվել են` ճանապար...,"Некоторые автобусные остановки отменены, чтобы...",NaN,NaN
4,summary,3,NaN,NaN,The following lines will have additional trips...,Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日、 週六及週日提供額外的出行服務： 2, 4, 16, 33, 45, 5...",Các tuyến sau đây sẽ có thêm chuyến vào các ng...,"다음 노선에는 주중, 토요일과 일요일에 운행을 증편할 예정입니다: 2, 4, 16,...","次の路線は、 平日、土曜日、日曜日の本数が追加されます: 2, 4, 16, 33, 45,...",Հետևյալ գծերը կունենան լրացուցիչ ուղևորություն...,По следующим маршрутам вводятся дополнительные...,NaN,NaN
5,summary,4,NaN,NaN,"WEEKDAYS, the following lines will have additi...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日提供 額外的出行服務： 10, 14, 28, 40, 51, 53, 7...","VÀO CÁC NGÀY TRONG TUẦN, những tuyến sau sẽ đư...","주중에 다음 노선은 운행을 증편할 예정입니다: 10, 14, 28, 40, 51, ...","平日は、次の路線の本数が 追加されます: 10, 14, 28, 40, 51, 53, 7...","ԱՇԽԱՏԱՆՔԱՅԻՆ ՕՐԵՐԻՆ, հետևյալ գծերը կունենան լր...",По БУДНЯМ по следующим маршрутам будут осущест...,NaN,NaN
6,summary,5,NaN,NaN,"SATURDAYS and SUNDAYS, the following lines hav...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在週六及週日提供 額外的出行服務： 154, 155, 210","VÀO THỨ BẢY và CHỦ NHẬT, những tuyến sau sẽ đư...","토요일과 일요일에 다음 노선은 운행을 증편할 예정입니다: 154, 155, 210","土曜日、日曜日は、 次の路線の本数が追加されます: 154, 155, 210","ՇԱԲԱԹ և ԿԻՐԱԿԻ օրերին, հետևյալ գծերն ունեն լրա...",По СУББОТАМ и ВОСКРЕСЕНЬЯМ вводятся дополнител...,NaN,NaN
31,summary,7,NaN,NaN,"B Line, D Line – Frequency peak periods weekda...",Línea B y línea D: La frecuencia de las horas ...,B 線路、D 線路 – 在工作日高峰期， Metro B線路（紅線）及D 線路（紫線）的發車...,"Tuyến B, Tuyến D – Giờ cao điểm các ngày trong...","B 노선, D 노선 – Metro B 노선(적색)과 D을 노선(자주색)을 대상으로 ...",B ライン 、D ライン - 平日のピーク時は、 Metro B ライン (レッドライン) ...,"B գիծ, D գիծ՝ Պիկ հաճախականությունը աշխատանքայ...","B линия , D линия - время ожидания в часы пик ...",NaN,NaN


### 2.2. Adding pre-header for `details`

In [73]:
detail_header = df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 20) & (df['en'].str.contains('We’re modify') == True)]

detail_header = detail_header.assign(section='details')
detail_header = detail_header.assign(order=0)

final_df = final_df.append(detail_header)
detail_header
# final_df.to_json('final_takeone.json',orient='records')

,en,es,zh-TW,ko,vi,ja,ru,hy,section,order
7,We’re modifying service on these bus lines:,Estamos modificando el servicio en las siguien...,我們正在變更以下巴士路線的 服務：,다음 버스 노선을 대상으로 버스편 운행 변경을 실시하고 있습니다:,Chúng tôi sẽ thay đổi dịch vụ trên những tuyến...,次のバス路線の サービスに変更があります:,Мы вносим изменения в схему движения следующих...,Մենք փոփոխում ենք ծառայությունը հետևյալ ավտոբո...,details,0


In [74]:
detail_header

,en,es,zh-TW,ko,vi,ja,ru,hy,section,order
7,We’re modifying service on these bus lines:,Estamos modificando el servicio en las siguien...,我們正在變更以下巴士路線的 服務：,다음 버스 노선을 대상으로 버스편 운행 변경을 실시하고 있습니다:,Chúng tôi sẽ thay đổi dịch vụ trên những tuyến...,次のバス路線の サービスに変更があります:,Мы вносим изменения в схему движения следующих...,Մենք փոփոխում ենք ծառայությունը հետևյալ ավտոբո...,details,0


### 2.3 Adding the `details`/lines section

#### 2.3.1 Process all the lines
First we will read all the lines in from the master list of all the lines.

In [75]:
lines_df = pd.read_csv('../data/input/mybus-sep-2021 - Lines.csv', index_col=0)
lines_df['AltLine'] = lines_df.AltLine.fillna(0).astype(int)
all_lines = lines_df[['Line Label',"AltLine"]]

lines_count = all_lines.shape[0]

all_lines['order'] = ''
all_lines = all_lines.sort_values(by="Line Number")
for i in range(0,lines_count):
    all_lines['order'].values[i] = i+1
all_lines.reset_index(inplace=True)
all_lines = all_lines.rename(columns={"Line Label":"line_label","Line Number":"line"})
all_lines.head(4)

<ipython-input-75-d0a7fe914b0c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_lines['order'] = ''


,line,line_label,AltLine,order
0,2,2,0,1
1,4,4,0,2
2,10,10,10,3
3,14,14,14,4


#### 2.3.2 Filter the docx table for the `line details`
 

In [76]:
### filter the lines out based on em-dash and rail lines
lines_takeone_df = df.loc[(df['en'].str.contains('\u2013')) & (df['en'].str.contains('B Line, D Line') == False)]

### create a field called `line` and set it to the first part of the split `em-dash`
lines_takeone_df['line'] = lines_takeone_df.en.str.split('–').str[0]

### extract duplicates
lines_takeone_df = lines_takeone_df.assign(oid=lines_takeone_df.line.str.split('/')).explode('oid')
dupes = lines_takeone_df.loc[(lines_takeone_df.duplicated(subset=['line']))]

### remove duplicates
lines_takeone_df = lines_takeone_df.drop_duplicates(subset=['line'])

### remove any lines with the "/" in it
lines_takeone_df = lines_takeone_df[lines_takeone_df["line"].str.contains("/")==False]

# lines_takeone_df
# lines_takeone_df

<ipython-input-76-3a1bc6085d78>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_takeone_df['line'] = lines_takeone_df.en.str.split('–').str[0]


In [77]:

# dupes2 = dupes.replace({'(\d+([ ]?[/])\d+)': '<br>'}, regex=True)
# dupes2

#### 2.3.3 Re-add duplicates

In [78]:
dupes['line'] = dupes['line'].str.split('/')
dupes = dupes.explode('line')
temp_df = dupes
temp_df2 = pd.DataFrame()
# dupes
for this_line in dupes['line']:
  line = this_line.strip(" ")
  print(line)
  temp_df = dupes[dupes["line"].str.contains(line)]
  temp_df = temp_df.replace({'(\d+([ ]?[/])\d+)': line}, regex=True,limit=1)
  temp_df = temp_df.replace({'(\d+)([ ][y][ ])(\d+)[ ]?:': line+":"}, regex=True,limit=1)
  temp_df = temp_df.replace({'Líneas': 'Línea'}, regex=True,limit=1)
  # temp_df = temp_df.replace({'(\d+([ ]?[-][ ]?)\d+)': line}, regex=True,limit=1)
  temp_df = temp_df.replace({'(\d+([ ][՝][ ])\d+)': line}, regex=True,limit=1)
  # temp_df = temp_df.replace({'(\d+([ ][՝][ ])\d+)': line}, regex=True,limit=1)
  temp_df2 = temp_df2.append(temp_df)

# temp_df
# lines_takeone_df = lines_takeone_df.append(dupes)
# this_line
# df_updated = dupes.replace({'|*****|': dupes['line']}, regex=True,limit=1)
  # Print the updated dataframe
# df_updated

lines_takeone_df = lines_takeone_df.append(temp_df2)
# df_updated

temp_df2
# dupes

242
243


,en,es,zh-TW,ko,vi,ja,ru,hy,line,oid
22,242 – Lines 242 & 243 will operate more freque...,Línea 242: Las líneas 242 y 243 operarán con m...,242 – 242號線及243號線將在工作日及週六提供更加頻繁的服務，並且在Ventura ...,242 – 주중과 토요일에 노선242 & 243 증편 운행 및 Ventura Bl와...,242 – Các Tuyến 242 và 243 sẽ được tăng chuyến...,242 - 242、243番路線は、平日、土曜日はVentura BlとDevonshire...,242 – Маршрут № 242 и 243 будет ходить чаще по...,242՝ 242 և 243 գծերը կգործեն ավելի հաճախակի աշ...,242,243
22,243 – Lines 242 & 243 will operate more freque...,Línea 243: Las líneas 242 y 243 operarán con m...,243 – 242號線及243號線將在工作日及週六提供更加頻繁的服務，並且在Ventura ...,243 – 주중과 토요일에 노선242 & 243 증편 운행 및 Ventura Bl와...,243 – Các Tuyến 242 và 243 sẽ được tăng chuyến...,243 - 242、243番路線は、平日、土曜日はVentura BlとDevonshire...,243 – Маршрут № 242 и 243 будет ходить чаще по...,243՝ 242 և 243 գծերը կգործեն ավելի հաճախակի աշ...,243,243


#### 2.3.4 Join pdfs

In [79]:
# import shutil
import os

#define the folders to look through
folders = os.listdir("../files/schedules")

#set an array for the file types
pdfs_list = []

#create a list of file types
for root, dirs, files in os.walk("../files/schedules"):
    for filename in files:
        lines = filename.replace(" ","").split("_TT")[0].split("-")
        for line in lines:
            this_schedule = {}
            this_schedule['line'] = line.lstrip("0")
            this_schedule['new-schedule'] = "./files/schedules/"+filename
            pdfs_list.append(this_schedule)
            # print(line)
# print(pdfs_list)

schedule_df = pd.DataFrame(pdfs_list)
schedule_df.tail(10)



,line,new-schedule
76,617,./files/schedules/617_TT_09-12-21.pdf
77,662,./files/schedules/662_TT_09-12-21.pdf
78,690,./files/schedules/690_TT_09-12-21.pdf
79,720,./files/schedules/720_TT_09-12-21.pdf
80,754,./files/schedules/754_TT_09-12-21.pdf
81,761,./files/schedules/761_TT_09-12-21.pdf
82,854,./files/schedules/854_TT_09-12-21.pdf
83,901,./files/schedules/901_TT_09-12-21.pdf
84,910,./files/schedules/910-950_TT_09-12-21.pdf
85,950,./files/schedules/910-950_TT_09-12-21.pdf


#### 2.3.5 Join `lines docx` data to `all lines` data

We use the pandas method `merge` to join the data on the `line` field and use an `outer` join to make sure to keep all the line data.

In [80]:
### convert the unique line field to the same data type, integers 
all_lines['line'] = all_lines['line'].astype(int)
lines_takeone_df['line'] = lines_takeone_df['line'].astype(int)
schedule_df['line'] = schedule_df['line'].astype(int)

### perform the merge 
merged_lines = all_lines.merge(lines_takeone_df, on='line',how='outer')
merged_lines2 = merged_lines.merge(schedule_df, on='line',how='outer')

### assign the "details" section
merged_lines2 = merged_lines2.assign(section='details')
# merged_lines['AltLine'] = all_lines['line'].astype(int)
merged_lines2

,line,line_label,AltLine,order,en,es,zh-TW,ko,vi,ja,ru,hy,oid,new-schedule,section
0,2,2,0,1,2 – No route changes. Bus stop consolidation.,Línea 2: Sin cambios de ruta. Consolidación de...,2 – 行駛路線無變更。巴士站整合。,2 – 노선 변경은 없음. 버스 정류소 통합.,2 – Không thay đổi lộ trình. Hợp nhất trạm dừn...,2 - ルートの変更なし。バス停の統合。,2 - Без изменений. Объединение автобусных оста...,2՝ Ոչ մի երթուղի չի փոխվում: Ավտոբուսի կանգառի...,2,./files/schedules/002_TT_09-12-21.pdf,details
1,4,4,0,2,4 – To create one high-frequency service for S...,Línea 4: Con el fin de crear un servicio de al...,4 – 為了在洛杉磯市中心和Santa Monica之間為Santa Monica Bl建設...,4 – LA 다운타운과 Santa Monica 구간을 연결하는 Santa Monic...,4 – Để xây dựng dịch vụ tần suất cao cho Santa...,4 - downtown LAとSanta Monicaの間のSanta Monica通りに...,4 - С целью создания единого маршрута с высоко...,4՝ Santa Monica Bl-ի համար մեկ բարձր հաճախական...,4,./files/schedules/004_TT_09-12-21.pdf,details
2,10,10,10,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./files/schedules/010_TT_09-12-21.pdf,details
3,14,14,14,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./files/schedules/014_TT_09-12-21.pdf,details
4,16,16,0,5,16 – New overnight Owl service.,Línea 16: Nuevo servicio nocturno.,16 – 新的通宵夜班車服務。,16 – 야간 Owl 버스편 신설.,16 – Dịch vụ Owl hoạt động xuyên đêm mới.,16 - 夜通し運行の新しい夜間サービス。,16 - Новый ночной маршрут.,16՝ Նոր գիշերային «Owl» ծառայություն:,16,./files/schedules/016_TT_09-12-21.pdf,details
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,761,761,0,121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./files/schedules/761_TT_09-12-21.pdf,details
121,854,854 / L Line (Gold) Shuttle,0,122,854 – Line 854 Gold Line Bus Bridge will be re...,Línea 854: La línea 854 se desviará debido a l...,854 – 因為洛杉磯市中心Little Tokyo地區在進行區域連接線建設，854號線Go...,"854 – Regional Connector 공사로 인한 1st St, Judge ...",854 – Tuyến 854 Gold Line Bus Bridge sẽ được đ...,854 - 854番路線 Gold Line Bus Bridgeは、downtown LA...,854 - В связи со строительством Регионального ...,854՝ Գիծ 854 Gold Line Bus Bridge կփոխվի Լոս Ա...,854,./files/schedules/854_TT_09-12-21.pdf,details
122,901,901 / G Line (Orange),0,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./files/schedules/901_TT_09-12-21.pdf,details
123,910,910 / J Line (Silver),910,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./files/schedules/910-950_TT_09-12-21.pdf,details


#### 2.3.6 Join the merged lines to the final data frame

In [81]:
final_df = final_df.append(merged_lines2)

#### 2.3.7 Join the rail data at the end of the `details` 

In [82]:
final_df.head(20)

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,new-schedule,current-schedule,line_label,AltLine,oid
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN,NaN,NaN,NaN,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN,NaN,NaN,NaN,NaN
1,summary,0,NaN,NaN,"Starting on Sunday, September 12, 2021, Metro ...","A partir del domingo, 12 de septiembre de 2021...",自2021年9月12日週日開始， Metro將對服務進行變更， 以便為您帶來更好的巴士 體驗。,"Bắt đầu từ Chủ Nhật, ngày 12 tháng 9 năm 2021,...","2021년 9월 12일, 일요일부터 개시, Metro가 향상된 버스 경험을 제공해 ...",2021年9月12日 日曜日より、 Metroは お客様により良いバスの旅をご体験いただける...,"2021 թվականի կիրակի, սեպտեմբերի 12-ից սկսած, M...","Начиная с субботы, 12 сентября 2021 года, Metr...",NaN,NaN,NaN,NaN,NaN
2,summary,1,NaN,NaN,We’re realigning routes to better match travel...,Estamos reorganizando las rutas para que se aj...,我們將重新調整行駛路線， 以讓其與乘車模式更加匹配， 並且會增加出行路線，以便實施 Next...,Chúng tôi sẽ điều chỉnh lại các tuyến lộ trình...,더 원할한 통행 패턴 반영을 위한 노선 재편과 운행 증편을 통해 NextGen 버스...,もっと旅行パターンに合わせたルートの再編成を行い、 次世代バスプラン (NextGen Bu...,Մենք փոփոխություններ ենք կատարում երթուղիներու...,"Мы изменяем схемы движения, чтобы привести их ...",NaN,NaN,NaN,NaN,NaN
3,summary,2,NaN,NaN,Some bus stops have been removed to improve tr...,Se eliminaron algunas paradas de autobús para ...,為縮短乘車時間已停用部分巴士站。,Một số trạm dừng xe buýt đã bị xóa bỏ để cải t...,운행 시간 개선을 위해 일부 버스 정류소를 폐쇄하였습니다.,いくつかのバス停は移動時間を改善するために撤去されました。,Որոշ ավտոբուսային կանգառներ հանվել են` ճանապար...,"Некоторые автобусные остановки отменены, чтобы...",NaN,NaN,NaN,NaN,NaN
4,summary,3,NaN,NaN,The following lines will have additional trips...,Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日、 週六及週日提供額外的出行服務： 2, 4, 16, 33, 45, 5...",Các tuyến sau đây sẽ có thêm chuyến vào các ng...,"다음 노선에는 주중, 토요일과 일요일에 운행을 증편할 예정입니다: 2, 4, 16,...","次の路線は、 平日、土曜日、日曜日の本数が追加されます: 2, 4, 16, 33, 45,...",Հետևյալ գծերը կունենան լրացուցիչ ուղևորություն...,По следующим маршрутам вводятся дополнительные...,NaN,NaN,NaN,NaN,NaN
5,summary,4,NaN,NaN,"WEEKDAYS, the following lines will have additi...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日提供 額外的出行服務： 10, 14, 28, 40, 51, 53, 7...","VÀO CÁC NGÀY TRONG TUẦN, những tuyến sau sẽ đư...","주중에 다음 노선은 운행을 증편할 예정입니다: 10, 14, 28, 40, 51, ...","平日は、次の路線の本数が 追加されます: 10, 14, 28, 40, 51, 53, 7...","ԱՇԽԱՏԱՆՔԱՅԻՆ ՕՐԵՐԻՆ, հետևյալ գծերը կունենան լր...",По БУДНЯМ по следующим маршрутам будут осущест...,NaN,NaN,NaN,NaN,NaN
6,summary,5,NaN,NaN,"SATURDAYS and SUNDAYS, the following lines hav...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在週六及週日提供 額外的出行服務： 154, 155, 210","VÀO THỨ BẢY và CHỦ NHẬT, những tuyến sau sẽ đư...","토요일과 일요일에 다음 노선은 운행을 증편할 예정입니다: 154, 155, 210","土曜日、日曜日は、 次の路線の本数が追加されます: 154, 155, 210","ՇԱԲԱԹ և ԿԻՐԱԿԻ օրերին, հետևյալ գծերն ունեն լրա...",По СУББОТАМ и ВОСКРЕСЕНЬЯМ вводятся дополнител...,NaN,NaN,NaN,NaN,NaN
31,summary,7,NaN,NaN,"B Line, D Line – Frequency peak periods weekda...",Línea B y línea D: La frecuencia de las horas ...,B 線路、D 線路 – 在工作日高峰期， Metro B線路（紅線）及D 線路（紫線）的發車...,"Tuyến B, Tuyến D – Giờ cao điểm các ngày trong...","B 노선, D 노선 – Metro B 노선(적색)과 D을 노선(자주색)을 대상으로 ...",B ライン 、D ライン - 平日のピーク時は、 Metro B ライン (レッドライン) ...,"B գիծ, D գիծ՝ Պիկ հաճախականությունը աշխատանքայ...","B линия , D линия - время ожидания в часы пик ...",NaN,NaN,NaN,NaN,NaN
7,details,0,NaN,NaN,We’re modifying service on these bus lines:,Estamos modificando el servicio en las siguien...,我們正在變更以下巴士路線的 服務：,Chúng tôi sẽ thay đổi dịch vụ trên những

### 2.4 Add the `end` section

In [83]:
### process the first end section
df = df.replace('metro.net/micro',r'<a href="https://www.metro.net/micro">metro.net/micro</a>',regex=True)
end1 = df.loc[(df['en'].str.contains('For more information '))]

end1 = end1.assign(section='end')
end1 = end1.assign(order=1)

### process the second end section
end2 = df.loc[(df['en'].str.contains('\\* M'))]

end2 = end2.assign(order=2)
end2 = end2.assign(section='end')

### add the second end section to the first
end1 = end1.append(end2)

### add the end section to the final data frame
final_df = final_df.append(end1)

### preview the end section
end1

,en,es,zh-TW,ko,vi,ja,ru,hy,section,order
32,For more information on Metro service changes ...,Para obtener más información sobre los cambios...,如需更多關於Metro服務變更及其他Metro服務的資訊，請透過致電323.GO.METRO...,Metro 버스편 변경과 기타 Metro 서비스에 관한 자세한 내용은 Metro 고...,Để biết thêm thông tin về những thay đổi trong...,Metroサービスの変更、その他のMetroサービスに関する詳細情報は、Metroカスタマー...,Для получения дополнительной информации об изм...,Metro յի սպասարկման փոփոխությունների և Metroյի...,end,1
33,* Metro Micro is Metro’s new on-demand ridesha...,*Metro Micro es el nuevo servicio de transport...,* Metro Micro是Metro的新 按需拼車服務， 在洛杉磯縣的幾個區域中提供出行服...,* Metro Micro는 Metro의 새로운 주문형 승차공유 서비스로 LA 카운티...,* Metro Micro là dịch vụ đi chung xe theo yêu ...,* Metro MicroはMetroの新しい オンデマンドの相乗りサービスで、 ロサンゼル...,"* Metro Micro - это новый сервис Metro, предос...",* Metro Micro-ն Metro-ի նոր ըստ պահանջի ուղևոր...,end,2


## Final output
### 3.1 Additional edits

In [84]:

final_df.loc[final_df.line==55, ['en', 'es', 'zh-TW', 'vi', 'ko', 'ja', 'hy', 'ru']] = 'New stop at Compton / 89th St for the southbound Line 55.'
final_df.loc[final_df.line==55]

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,new-schedule,current-schedule,line_label,AltLine,oid
18,details,19,55,NaN,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,New stop at Compton / 89th St for the southbou...,./files/schedules/055_TT_09-12-21.pdf,NaN,55,0.0,NaN


### 3.2 Check the data frame

In [85]:
final_df.head(55)

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,new-schedule,current-schedule,line_label,AltLine,oid
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN,NaN,NaN,NaN,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN,NaN,NaN,NaN,NaN
1,summary,0,NaN,NaN,"Starting on Sunday, September 12, 2021, Metro ...","A partir del domingo, 12 de septiembre de 2021...",自2021年9月12日週日開始， Metro將對服務進行變更， 以便為您帶來更好的巴士 體驗。,"Bắt đầu từ Chủ Nhật, ngày 12 tháng 9 năm 2021,...","2021년 9월 12일, 일요일부터 개시, Metro가 향상된 버스 경험을 제공해 ...",2021年9月12日 日曜日より、 Metroは お客様により良いバスの旅をご体験いただける...,"2021 թվականի կիրակի, սեպտեմբերի 12-ից սկսած, M...","Начиная с субботы, 12 сентября 2021 года, Metr...",NaN,NaN,NaN,NaN,NaN
2,summary,1,NaN,NaN,We’re realigning routes to better match travel...,Estamos reorganizando las rutas para que se aj...,我們將重新調整行駛路線， 以讓其與乘車模式更加匹配， 並且會增加出行路線，以便實施 Next...,Chúng tôi sẽ điều chỉnh lại các tuyến lộ trình...,더 원할한 통행 패턴 반영을 위한 노선 재편과 운행 증편을 통해 NextGen 버스...,もっと旅行パターンに合わせたルートの再編成を行い、 次世代バスプラン (NextGen Bu...,Մենք փոփոխություններ ենք կատարում երթուղիներու...,"Мы изменяем схемы движения, чтобы привести их ...",NaN,NaN,NaN,NaN,NaN
3,summary,2,NaN,NaN,Some bus stops have been removed to improve tr...,Se eliminaron algunas paradas de autobús para ...,為縮短乘車時間已停用部分巴士站。,Một số trạm dừng xe buýt đã bị xóa bỏ để cải t...,운행 시간 개선을 위해 일부 버스 정류소를 폐쇄하였습니다.,いくつかのバス停は移動時間を改善するために撤去されました。,Որոշ ավտոբուսային կանգառներ հանվել են` ճանապար...,"Некоторые автобусные остановки отменены, чтобы...",NaN,NaN,NaN,NaN,NaN
4,summary,3,NaN,NaN,The following lines will have additional trips...,Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日、 週六及週日提供額外的出行服務： 2, 4, 16, 33, 45, 5...",Các tuyến sau đây sẽ có thêm chuyến vào các ng...,"다음 노선에는 주중, 토요일과 일요일에 운행을 증편할 예정입니다: 2, 4, 16,...","次の路線は、 平日、土曜日、日曜日の本数が追加されます: 2, 4, 16, 33, 45,...",Հետևյալ գծերը կունենան լրացուցիչ ուղևորություն...,По следующим маршрутам вводятся дополнительные...,NaN,NaN,NaN,NaN,NaN
5,summary,4,NaN,NaN,"WEEKDAYS, the following lines will have additi...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日提供 額外的出行服務： 10, 14, 28, 40, 51, 53, 7...","VÀO CÁC NGÀY TRONG TUẦN, những tuyến sau sẽ đư...","주중에 다음 노선은 운행을 증편할 예정입니다: 10, 14, 28, 40, 51, ...","平日は、次の路線の本数が 追加されます: 10, 14, 28, 40, 51, 53, 7...","ԱՇԽԱՏԱՆՔԱՅԻՆ ՕՐԵՐԻՆ, հետևյալ գծերը կունենան լր...",По БУДНЯМ по следующим маршрутам будут осущест...,NaN,NaN,NaN,NaN,NaN
6,summary,5,NaN,NaN,"SATURDAYS and SUNDAYS, the following lines hav...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在週六及週日提供 額外的出行服務： 154, 155, 210","VÀO THỨ BẢY và CHỦ NHẬT, những tuyến sau sẽ đư...","토요일과 일요일에 다음 노선은 운행을 증편할 예정입니다: 154, 155, 210","土曜日、日曜日は、 次の路線の本数が追加されます: 154, 155, 210","ՇԱԲԱԹ և ԿԻՐԱԿԻ օրերին, հետևյալ գծերն ունեն լրա...",По СУББОТАМ и ВОСКРЕСЕНЬЯМ вводятся дополнител...,NaN,NaN,NaN,NaN,NaN
31,summary,7,NaN,NaN,"B Line, D Line – Frequency peak periods weekda...",Línea B y línea D: La frecuencia de las horas ...,B 線路、D 線路 – 在工作日高峰期， Metro B線路（紅線）及D 線路（紫線）的發車...,"Tuyến B, Tuyến D – Giờ cao điểm các ngày trong...","B 노선, D 노선 – Metro B 노선(적색)과 D을 노선(자주색)을 대상으로 ...",B ライン 、D ライン - 平日のピーク時は、 Metro B ライン (レッドライン) ...,"B գիծ, D գիծ՝ Պիկ հաճախականությունը աշխատանքայ...","B линия , D линия - время ожидания в часы пик ...",NaN,NaN,NaN,NaN,NaN
7,details,0,NaN,NaN,We’re modifying service on these bus lines:,Estamos modificando el servicio en las siguien...,我們正在變更以下巴士路線的 服務：,Chúng tôi sẽ thay đổi dịch vụ trên những

### 3.2 Split the final data frame into JSON files depending on the language

In [86]:
languages = ['en','es','zh-TW','vi','ko','ja','hy','ru']
DATA_OUTPUT_PATH = "../data/takeones/"
for i in languages:
    final_final_df = final_df[['section','order', i,'line', 'new-schedule', 'current-schedule']].copy()
    final_final_df = final_final_df.rename(columns={i: 'content'})
    final_final_df.to_json(DATA_OUTPUT_PATH + 'takeone-' + i + '.json',orient='records')
    print('Takeone created for: ' + i)

Takeone created for: en
Takeone created for: es
Takeone created for: zh-TW
Takeone created for: vi
Takeone created for: ko
Takeone created for: ja
Takeone created for: hy
Takeone created for: ru


## Extra code

In [87]:
### RIP: code to split based on `:`
# th['en'] = th['en'].str.split(':')
# th = th.explode('en')
###